In [1]:
import os
import uuid # For creating Universally unique identifier
import pickle
import pandas as pd
import mlflow
from typing import List

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [7]:
year = 2021
month = 3
taxi_type = "green"
input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"output/{taxi_type}/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
os.makedirs(os.path.split(output_file)[0], exist_ok=True)

RUN_ID = os.getenv("RUN_ID", "10f4197008104ad183466cdb19e26c4e")

In [8]:
def generate_uuid(n: int) -> pd.DataFrame:
    ride_ids = [str(uuid.uuid4()) for _ in range(n)]
    return ride_ids

def read_dataframe(filename: str) -> pd.DataFrame:
    df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df["duration"] = df["duration"].dt.total_seconds() / 60
    df = df[(df["duration"] >= 1) & (df["duration"] <= 60)]
    df["ride_id"] = generate_uuid(len(df))

    return df

def prepare_dictionaries(df: pd.DataFrame) -> List[dict]:
    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")
    return dicts

In [9]:
def load_model(run_id):
    logged_model = f"s3://mlflow-artifacts-remote-jw/1/{run_id}/artifacts/model/"
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)

    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result["rid_id"] = df["ride_id"]
    df_result["lpep_pickup_datetime"] = df["lpep_pickup_datetime"]
    df_result["PULocationID"] = df["PULocationID"]
    df_result["DOLocationID"] = df["DOLocationID"]
    df_result["actual_duration"] = df["duration"]
    df_result["predicted_duration"] = y_pred
    df_result["diff"] = df_result["actual_duration"] - df_result["predicted_duration"]
    df_result["model_version"] = run_id

    df_result.to_parquet(output_file, index=False)
        

In [10]:
apply_model(
    input_file=input_file,
    run_id=RUN_ID,
    output_file=output_file
)

In [11]:
!ls output/green/

green_tripdata_2021-03.parquet
